# 🚀 From Model to Production with MLflow – KI Summer Summit Edition

## 🎯 Ziel des Workshops
- Einfaches Machine-Learning-Modell trainieren.
- Trainingsversuche mit **MLflow** tracken.
- Bestes Modell in der **MLflow Model Registry** speichern und versionieren.
- Modell als Test-API mit **Docker + MLflow** bereitstellen.

## 🤔 Warum ist das wichtig?
- Schwierigkeit ohne MLOps:
  - Überblick über Hyperparameter und Datenvarianten geht verloren.
  - Gute Modelle wiederfinden ist mühsam.
  - Weitergabe an Kollegen oft chaotisch (`modell_final_v2_wirklich_final.pkl`).
- **MLOps** schafft Professionalität und Struktur – ähnlich wie DevOps.

## 🔧 Vorteile von MLflow

### 1. Experiment Tracking
- Erfasst Parameter, Metriken, Artefakte, Code-Versionen.
- Vergleich und Analyse von Experimenten.
- Unterstützt Reproduzierbarkeit und Hyperparameter-Tuning.

### 2. Model Packaging (MLflow Models)
- Einheitliches Format (`MLmodel`).
- Plattformübergreifende Nutzung (Python, R, Docker, REST, Spark).
- Standardisierte Modellübergabe.
- Kompatibel mit Scikit-learn, TensorFlow, PyTorch u. v. m.

### 3. Model Registry
- Zentrale Verwaltung und Versionierung von Modellen.
- **Stages**: `Staging`, `Production`, `Archived`.
- Dokumentation mit **Kommentaren** und **Tags**.
- Unterstützt Review, Freigabe und CI/CD.

### 4. Deployment & Reproduzierbarkeit
- Deployment auf REST-API, Docker, Azure ML, AWS SageMaker, Kubernetes.
- Logging von Umgebung, Datenpfaden und Code-Versionen.
- Sicherstellung von Reproduzierbarkeit auch langfristig.

## 💡 Warum MLflow?
- 🔄 Reproduzierbare ML-Prozesse.
- 👥 Bessere Team-Zusammenarbeit durch zentrale Modellverwaltung.
- ⚙️ Struktur statt Chaos.
- ⏱️ Schnelleres Deployment in Produktion.
- 📊 Mehr Fokus auf Modellqualität statt Dokumentation.


---
## 📊 2. Vorbereitung: Notwendige Werkzeuge installieren

In [ ]:
!pip install mlflow scikit-learn pandas matplotlib seaborn openpyxl -q
print("Pakete installiert!")

In [ ]:
print("Die installierten Werkzeuge laden")
import mlflow
import mlflow.sklearn # Speziell für Scikit-learn Modelle mit MLflow
import pandas as pd # Für Tabellen-Daten (wie Excel, aber in Python)
import matplotlib.pyplot as plt # Zum Erstellen von Diagrammen
import seaborn as sns # Macht Diagramme noch schöner und einfacher
from sklearn.model_selection import train_test_split # Zum Aufteilen unserer Daten
from sklearn.ensemble import RandomForestClassifier # Unser KI-Modell für heute
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay # Zum Bewerten des Modells
from sklearn.datasets import load_iris # Ein bekannter Beispieldatensatz
import os # Um mit Ordnern und Dateien zu arbeiten
import json # Um Daten im JSON-Format zu verarbeiten (wichtig für APIs)
import requests # Um Anfragen an Web-APIs zu senden
import sklearn # Um die Version von Scikit-learn zu prüfen

# Diese Zeile sorgt dafür, dass Diagramme direkt hier im Notebook angezeigt werden
%matplotlib inline

print(f"MLflow Version: {mlflow.__version__}")
print(f"Scikit-learn Version: {sklearn.__version__}")
print(f"Pandas Version: {pd.__version__}")

### 🔌 Verbindung zum gemeinsamen MLflow Tracking Server
Wir verwenden einen gemeinsamen MLflow Server. Damit wir eure Experimente auseinanderhalten können, erstellt bitte jeder sein eigenes Experiment mit seinem Namen.

**WICHTIG:** Ändere `DEIN_NAME_HIER` in der nächsten Zelle zu deinem Namen oder einem eindeutigen Kürzel (ohne Leerzeichen, am besten nur Buchstaben, Zahlen, Unterstriche).

In [ ]:
# Dein Name oder ein eindeutiges Kürzel (z.B. MaxM, AnnaS)
# BITTE ÄNDERN:
TEILNEHMER_NAME = "Ahmad" # <<< HIER DEINEN NAMEN EINTRAGEN!

if TEILNEHMER_NAME == "DEIN_NAME_HIER" or TEILNEHMER_NAME == "":
    print("WARNUNG: Bitte setze die Variable TEILNEHMER_NAME auf deinen Namen oder ein Kürzel!")

os.environ["MLFLOW_TRACKING_USERNAME"] = "mlflow"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "SprinteinsMlFlow123#"

# Die Adresse unseres gemeinsamen MLflow Servers
MLFLOW_TRACKING_SERVER_URI = "https://mlflow.kaywan.de"

try:
    mlflow.set_tracking_uri(MLFLOW_TRACKING_SERVER_URI)
    print(f"Verbunden mit MLflow Tracking Server: {mlflow.get_tracking_uri()}")
except Exception as e:
    print(f"Konnte MLflow Tracking URI nicht setzen: {e}.")

# Eindeutiger Experimentname für dich
EXPERIMENT_NAME = f"Iris_Challenge_{TEILNEHMER_NAME}"
try:
    # Versucht, das Experiment zu erstellen oder auszuwählen, falls es schon existiert
    current_experiment = mlflow.get_experiment_by_name(EXPERIMENT_NAME)
    if current_experiment is None: # Experiment existiert nicht, neu erstellen
        experiment_id = mlflow.create_experiment(EXPERIMENT_NAME)
        print(f"Neues MLflow Experiment '{EXPERIMENT_NAME}' (ID: {experiment_id}) erstellt.")
    else: # Experiment existiert bereits
        experiment_id = current_experiment.experiment_id
        print(f"Vorhandenes MLflow Experiment '{EXPERIMENT_NAME}' (ID: {experiment_id}) ausgewählt.")
    mlflow.set_experiment(experiment_id=experiment_id) # Setze als aktives Experiment
    
    print(f"Dein Experiment in der MLflow UI: {MLFLOW_TRACKING_SERVER_URI}/#/experiments/{experiment_id}")
    
except Exception as e:
    print(f"Fehler beim Erstellen/Setzen des Experiments '{EXPERIMENT_NAME}': {e}")
    print("Stelle sicher, dass der Server erreichbar ist und dein TEILNEHMER_NAME gültig ist (und nicht leer oder der Platzhalter ist).")

---
## 🌱 3. Daten laden und verstehen
Jedes KI-Modell braucht Daten zum Lernen. Wir verwenden den "Iris"-Datensatz. Das ist ein klassischer Datensatz in der KI-Welt, der Merkmale von Schwertlilien (Iris) enthält. Ziel ist es, anhand der Blütenblatt- und Kelchblattmaße die Art der Iris vorherzusagen.

Zuerst laden wir die Daten und schauen sie uns kurz an.

In [ ]:
# Lade den Iris-Datensatz. 'as_frame=True' gibt uns eine schöne Tabelle (DataFrame).
iris_data = load_iris(as_frame=True)
df_iris = iris_data.frame

# Wir fügen eine Spalte hinzu, die den Namen der Iris-Art enthält, statt nur einer Zahl.
df_iris['target_name'] = df_iris['target'].map({0: 'Setosa', 1: 'Versicolor', 2: 'Virginica'})

print("So sehen die ersten 5 Zeilen unserer Daten aus:")
display(df_iris.head()) # 'display' ist schöner für Tabellen im Notebook

print(f"\nInsgesamt haben wir {df_iris.shape[0]} Datenpunkte (Blumen) und {df_iris.shape[1]} Spalten (Merkmale + Ziel). ")

### 🔎 Ein Bild sagt mehr als tausend Zahlen: Daten visualisieren

In [ ]:
print("Erstelle Pairplot... Das kann einen Moment dauern.")
plt.figure(figsize=(10, 8))

# Create a pairplot showing relationships between all iris featuresrelationships
sns.pairplot(df_iris, hue='target_name', diag_kind='kde', markers=["o", "s", "D"])
plt.suptitle("Beziehungen der Merkmale im Iris-Datensatz", y=1.02)
plt.show()

print("Man sieht schon: Manche Arten (z.B. Setosa, oft blau in Plots) scheinen sich gut von den anderen unterscheiden zu lassen!")

### Interpretation des Pairplots

- **Jede Grafik** zeigt die Beziehung zwischen zwei Merkmalen.
- **Diagonale Plots**: Verteilung jedes Merkmals pro Iris-Art.
- **Wichtige Beobachtungen**:
  - Die drei Iris-Arten (Setosa, Versicolor, Virginica) sind farblich und symbolisch unterschieden.
  - *Setosa* ist klar von den anderen Arten getrennt (besonders bei **petal length** und **petal width**).
  - *Versicolor* und *Virginica* überlappen teilweise → Klassifikation hier schwieriger.
  - **Blütenblattmerkmale (petal)** sind aussagekräftiger als **Kelchblattmerkmale (sepal)**.
- **Fazit**: Ein gutes ML-Modell sollte diese Muster erkennen und für die Klassifikation nutzen.


### 🎯 Was soll das Modell lernen? Features (X) und Target (y)
Wir müssen dem Modell sagen, welche Spalten die Eingabemerkmale (Features, oft `X` genannt) sind und welche Spalte das ist, was wir vorhersagen wollen (Target, oft `y` genannt).

In [ ]:
# Die Features sind die Spalten mit den Messwerten der Blumen.
X = df_iris[iris_data.feature_names] # iris_data.feature_names enthält die Namen der Merkmalsspalten

# Das Target ist die Spalte 'target', die die Art der Blume als Zahl (0, 1 oder 2) enthält.
y = df_iris['target']

print("Das sind unsere Eingabemerkmale (X) mit den zugehörigen Zielwerten (y):")

# Beispiele für Setosa (Target 0)
print("\n--- Beispiele für Setosa (Target 0) ---")
setosa_examples = pd.concat([X.iloc[:3], y.iloc[:3].rename("target")], axis=1)
display(setosa_examples)

# Beispiele für Versicolor (Target 1)
print("\n--- Beispiele für Versicolor (Target 1) ---")
versicolor_examples = pd.concat([X.iloc[50:53], y.iloc[50:53].rename("target")], axis=1)
display(versicolor_examples)

# Beispiele für Virginica (Target 2)
print("\n--- Beispiele für Virginica (Target 2) ---")
virginica_examples = pd.concat([X.iloc[100:103], y.iloc[100:103].rename("target")], axis=1)
display(virginica_examples)

print("\nUnser Modell wird diese Merkmale nutzen, um die Iris-Art (Target 0, 1 oder 2) vorherzusagen.")


---
## 🧠 4. Ein KI-Modell trainieren

- **Ziel:** Ein Klassifikationsmodell auf Basis von `RandomForestClassifier` trainieren.
- **Warum Random Forest?**  
  - Robust und oft sehr performant für Klassifikationsaufgaben.
  - Gut geeignet für kleine bis mittelgroße Datensätze.

In [ ]:
# Wir teilen die Daten: 80% zum Trainieren, 20% zum Testen.
# Warum teilen? 
# Modell soll nicht nur Trainingsdaten auswendig lernen (Vermeidung von **Overfitting**). 
# Wir brauchen Daten, die das Modell noch nicht gesehen hat (**Testset**), um die echte Vorhersagefähigkeit zu prüfen.
# Aufteilung:
# Trainingsset** → Modell lernen lassen.  
# Testset** → Modellleistung überprüfen.

# 'random_state=42' sorgt dafür, dass die Aufteilung immer gleich ist, wenn wir den Code nochmal ausführen.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42, stratify=y)

print(f"Wir haben {X_train.shape[0]} Datenpunkte zum Trainieren und {X_test.shape[0]} zum Testen.")

### 🤖 Das Modell lernt: Training des RandomForestClassifiers
Ein Modell hat verschiedene "Stellschrauben", sogenannte Hyperparameter. Diese beeinflussen, wie das Modell lernt. Wir wählen hier ein paar typische Werte.

In [ ]:
# Das sind unsere 'Stellschrauben' (Hyperparameter) für den RandomForest
model_hyperparameters = {
    "n_estimators": 100,     # Wie viele "Bäume" soll unser Wald haben? Weniger für schnelleres Training.
    "max_depth": 30,         # Wie tief darf jeder Baum werden?
    "random_state": 42      # Damit das Modelltraining reproduzierbar ist.
}

# Wir erstellen unser Modell mit diesen Einstellungen
rf_model = RandomForestClassifier(**model_hyperparameters)

# Jetzt trainieren wir das Modell mit unseren Trainingsdaten.
# Das Modell lernt jetzt Muster in X_train, um y_train vorhersagen zu können.
print("Modelltraining startet...")
rf_model.fit(X_train, y_train)
print("Modelltraining abgeschlossen!")

# Nun testen wir, wie gut unser Modell auf den *unbekannten* Testdaten ist.
y_predictions = rf_model.predict(X_test)

# Wir berechnen die Genauigkeit (Accuracy): Wie viel Prozent hat das Modell richtig vorhergesagt?
accuracy = accuracy_score(y_test, y_predictions)
print(f"\nGenauigkeit des Modells auf den Testdaten: {accuracy:.2f} (d.h. {accuracy*100:.0f}% richtig!)")

# Die Confusion Matrix zeigt uns genauer, wo das Modell Fehler gemacht hat.
print("\nConfusion Matrix (Zeilen: Echte Arten, Spalten: Vorhergesagte Arten):")
cm = confusion_matrix(y_test, y_predictions)
print(cm)

# Visualisieren der Confusion Matrix
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=iris_data.target_names)
fig, ax = plt.subplots(figsize=(6,5))
disp.plot(cmap=plt.cm.Blues, ax=ax)
ax.set_title("Leistung des Modells (Confusion Matrix)")
plt.show()

print("Eine perfekte Diagonale von links oben nach rechts unten wäre ideal.")

## **Zwischenfazit & Herausforderung**
- Modell erfolgreich trainiert → erste gute Ergebnisse.
- Aber: Änderungen der Hyperparameter (z. B. `n_estimators=100`, `max_depth=6`) beeinflussen das Ergebnis.
- Problem ohne System:
  - Manuelle Notizen nötig.
  - Ergebnisse schwer vergleichbar.
- **Lösung:** MLflow für strukturiertes Experiment-Tracking.

In [60]:
HTML('<img src="https://raw.githubusercontent.com/asalah6/mlops-workshop/refs/heads/main/ex_track.png" width="800" height="400" alt="MLflow Tracking Visualization">')



---

## 📦 **5. MLflow Tracking: Ordnung ins Experimentier-Chaos bringen**
- **MLflow-Funktionen:**
  - Speichert verwendete **Hyperparameter**.
  - Zeichnet erzielte **Metriken** auf (z. B. Accuracy).
  - Ermöglicht Ablage von **Artefakten**:
    - Diagramme (z. B. Confusion Matrix).
    - Trainiertes Modell.
- **Vorteile:**
  - Einfacher Vergleich verschiedener Trainingsläufe.
  - Schnelles Wiederfinden guter Modelle.
  - Grundlage für reproduzierbare Experimente.


In [ ]:
# Wir starten einen "MLflow Run". Das ist wie eine einzelne Aufzeichnungssitzung.
run_description = f"RF n_est={model_hyperparameters['n_estimators']}, depth={model_hyperparameters['max_depth']}"

# Der Name, unter dem wir das Modell später in der "Modell-Bibliothek" (Registry) finden wollen.
REGISTERED_MODEL_NAME = f"IrisRF_{TEILNEHMER_NAME}"

print(f"Starte MLflow Run für Experiment '{EXPERIMENT_NAME}'...")

try:
    # 'with' sorgt dafür, dass der Run am Ende automatisch geschlossen wird.
    with mlflow.start_run(run_name=run_description) as run:
        run_id = run.info.run_id
        print(f"MLflow Run gestartet mit ID: {run_id}")

        # 1. Parameter loggen (unsere "Stellschrauben")
        print("Logge Hyperparameter...")
        mlflow.log_params(model_hyperparameters)
        
        mlflow.log_param("daten_aufteilung", f"{X_train.shape[0]}/{X_test.shape[0]} train/test")
        mlflow.log_param("dataset_size", len(df_iris))
        mlflow.log_param("features_used", ", ".join(X.columns.tolist()))

        # 2. Metriken loggen (unsere Ergebnisse)
        print("Logge Metriken...")
        mlflow.log_metric("accuracy", accuracy)

        # 3. Artefakte loggen (z.B. Diagramme)
        print("Logge Confusion Matrix als Bild...")
        fig_cm, ax_cm = plt.subplots()
        cm_display = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=iris_data.target_names)
        cm_display.plot(ax=ax_cm, cmap=plt.cm.Greens)
        ax_cm.set_title("Confusion Matrix (geloggt mit MLflow)")
        # Speichern und loggen des Bildes
        mlflow.log_figure(fig_cm, "evaluation_plots/confusion_matrix.png")
        plt.close(fig_cm) # Schließen, damit es nicht doppelt angezeigt wird

        # 4. Das Modell selbst loggen UND in der Registry registrieren
        print(f"Logge das trainierte Modell und registriere es als '{REGISTERED_MODEL_NAME}'...")
        
        # 'input_example' und 'signature' helfen MLflow zu verstehen, welche Art von Daten das Modell erwartet und liefert.
        # Das ist nützlich für das spätere Bereitstellen des Modells als API.
        input_example = X_train.head(3) # Ein kleines Beispiel der Eingabedaten
        signature = mlflow.models.infer_signature(X_train, y_predictions) # MLflow versucht, das Format selbst zu erkennen
        
        mlflow.sklearn.log_model(
            sk_model=rf_model, # Das trainierte Modellobjekt
            artifact_path="rf_iris_model_files", # Ordnername innerhalb des Runs für die Modelldateien
            registered_model_name=REGISTERED_MODEL_NAME, # Name für die Model Registry
            signature=signature,
            input_example=input_example
        )
        print("Modell erfolgreich geloggt und registriert!")

    print("\nMLflow Run abgeschlossen.")
    print(f"Schau dir diesen Run in der MLflow UI an! (Experiment: '{EXPERIMENT_NAME}', Run ID: {run_id})")
    # print(f"Link zum Run (falls UI erreichbar): {mlflow.get_tracking_uri().replace('file:', '')}/#/experiments/{mlflow.get_experiment_by_name(EXPERIMENT_NAME).experiment_id}/runs/{run_id}")

except Exception as e:
    print(f"Ein Fehler ist im MLflow Run aufgetreten: {e}")
    print("Stelle sicher, dass der MLflow Tracking Server (oder der lokale Pfad) korrekt eingerichtet ist.")

## 📁 6. MLflow Model Registry – Zentrale Modellverwaltung

### **Was ist die Model Registry?**
- Zentraler Ort für gespeicherte **beste Modelle**.
- **Versionierung** jeder Modellvariante.
- **Stages** für unterschiedliche Zustände:
  - `None` / `Development` → In Entwicklung
  - `Staging` → Zum Testen
  - `Production` → Produktiv eingesetzt
  - `Archived` → Veraltet

---

### **Warum ist das wichtig?**
| Vorteil              | Nutzen für MLOps                             |
|----------------------|----------------------------------------------|
| 🤝 Zusammenarbeit     | Gemeinsamer Zugriff für Teams               |
| 🔁 Reproduzierbarkeit | Exakte Versionen für Analysen & Deployments |
| 🚀 Deployment         | Klare Prozesse für Übergabe & Freigabe      |
| 🧾 Governance         | Audit-Trail und Kontrolle produktiver Modelle |

---

### **Kernfunktionen**
1. **Versionierung**
   - Jede Registrierung eines Modells erzeugt eine neue Version.
   - Nachvollziehbarkeit bei Feintuning oder Datenupdates.
2. **Stage Management**
   - Definierte Übergänge: Development → Staging → Production → Archived.
   - Unterstützt Review- und Deployment-Workflows.
3. **Annotation & Dokumentation**
   - Kommentare, Metadaten und Beschreibungen hinzufügen.
   - Ermöglicht klare Nachvollziehbarkeit und Governance.
4. **Zentrale Modellbibliothek**
   - Alle Modelle an einem Ort auffindbar und dokumentiert.
   - Fördert Teamarbeit und Wiederverwendbarkeit.

---

### **Manuelle Registrierung (MLflow UI)**
1. **Besten Run auswählen** (nach Accuracy sortieren).
2. In den Run-Details → **Artifacts → Modellordner** (mit `MLmodel`).
3. **Register Model** anklicken.
4. **Neues Modell** erstellen (`IrisRF_DEIN_NAME`) oder bestehendes Modell auswählen.
5. **Description** hinzufügen → **Register** klicken.
6. Unter **"Models"** → Modell auswählen → Version ansehen.
7. **Stage ändern** (z. B. zu `Staging` oder `Production`).

---

**Ergebnis:**  
→ Dein bestes Modell ist versioniert, dokumentiert und bereit für den Einsatz.


---
## 🌐 7. Das Modell bereitstellen: Von der Datei zur API
Ein trainiertes Modell ist schön, aber meistens wollen wir es auch nutzen können, z.B. in einer App oder auf einer Webseite. Dafür stellen wir es oft als eine Art "Web-Service" (API) bereit. Man schickt Daten an die API, und sie schickt eine Vorhersage zurück.

MLflow bietet Werkzeuge, um das relativ einfach zu machen.

**WICHTIG:** Die folgenden Befehle sind für die **Kommandozeile (Terminal)** gedacht, nicht direkt für dieses Notebook. Du müsstest sie auf deinem Computer ausführen, wo MLflow und Docker (für Option 2) installiert sind.

### 7.1 Schneller Test: Modell lokal als API starten mit `mlflow models serve`
Dieser Befehl startet einen einfachen Webserver, der unser Modell über eine API ansprechbar macht. Perfekt für schnelle Tests.

**Beispiel-Kommando (im Terminal ausführen):**
```bash
# Zuerst: Sag dem Terminal, wo dein MLflow Server/Logbuch ist.
export MLFLOW_TRACKING_URI="https://mlflow.kaywan.de"
export MLFLOW_TRACKING_USERNAME="mlflow"
export MLFLOW_TRACKING_PASSWORD="SprinteinsMlFlow123#"

# Starte den Server für Version 1 unseres Modells auf Port 5001
export MODEL_NAME="IrisRF_Ahmad"
mlflow models serve -m "models:/${MODEL_NAME}/1" -p 5001 --env-manager local
#!nohup mlflow models serve -m "models:/${MODEL_NAME}/1" -p 5001 --env-manager local > mlflow.log 2>&1 &
#!ps aux | grep mlflow
#!kill 12345

# Alternativ, wenn du das Modell in Stage "Production" geschoben hast:
# mlflow models serve -m "models:/Irisblumen_Klassifikator_RF/Production" -p 5001 --env-manager local
```
-   `-m "models:/Modellname/VersionOderStage"`: Sagt MLflow, welches Modell es laden soll.
-   `-p 5001`: Der Netzwerk-Port, unter dem die API erreichbar ist.
-   `--env-manager local`: Nutzt deine lokale Python-Umgebung (einfacher für Demos).

### 7.2 Für die "echte Welt": Docker-Container erstellen mit `mlflow models build-docker`
Für eine robustere Bereitstellung packt man das Modell und alles, was es braucht, in einen **Docker-Container**, die überall gleich läuft.

**Beispiel-Kommando (im Terminal ausführen, Docker muss installiert sein):**
```bash

# Docker-Image für unser Modell erstellen (Name: iris-api-service)
mlflow models build-docker -m "models:/Irisblumen_Klassifikator_RF/1" -n "iris-api-service" --env-manager local

# Den erstellten Container starten (Port 5001 auf deinem PC leitet zu Port 8080 im Container)
# docker run -p 5001:8080 iris-api-service
```
Dieser Docker-Container könnte dann z.B. in der Cloud oder auf eigenen Servern betrieben werden.

### 7.3 Die API testen: Eine Anfrage senden
Wenn dein Modell-Server läuft (entweder mit `mlflow models serve` oder via Docker), können wir ihm jetzt Daten schicken und eine Vorhersage bekommen. Das machen wir hier im Notebook mit der `requests` Bibliothek.

In [ ]:
# Wir nehmen eine Beispiel-Blume aus unseren Testdaten
if 'X_test' in locals() and not X_test.empty:
    sample_flower_features = X_test.iloc[[0]] # Die erste Blume aus dem Testset
    print("Diese Blumendaten schicken wir an die API:")
    display(sample_flower_features)

    # Die API erwartet die Daten in einem bestimmten JSON-Format.
    # 'dataframe_split' ist ein gängiges Format.
    payload = {
        "dataframe_split": {
            "columns": sample_flower_features.columns.tolist(),
            "data": sample_flower_features.values.tolist()
        }
    }

    # Die Adresse unserer API (Port 5001, wie oben im Terminal-Befehl festgelegt)
    api_url = "http://localhost:5001/invocations"

    print(f"\nSende Anfrage an: {api_url}")
    print("Mit diesen Daten (JSON-Format):\n", json.dumps(payload, indent=2))

    try:
        # Wir senden die Daten an die API und warten auf die Antwort
        response = requests.post(api_url, json=payload, headers={"Content-Type": "application/json"})
        response.raise_for_status() # Löst einen Fehler aus, wenn die API ein Problem meldet (z.B. Fehler 400 oder 500)
        
        predictions_json = response.json()
        print("\nAntwort von der API (Status {}):".format(response.status_code))
        print(json.dumps(predictions_json, indent=2))
        
        # Die Vorhersage ist meistens unter dem Schlüssel 'predictions'
        if 'predictions' in predictions_json and len(predictions_json['predictions']) > 0:
            predicted_index = predictions_json['predictions'][0]
            # Umwandlung des Index (0, 1, 2) in den Namen der Iris-Art
            predicted_species_name = iris_data.target_names[int(predicted_index)]
            print(f"\nDas Modell sagt voraus: Es ist eine Iris '{predicted_species_name}' (Index {predicted_index}).")
        else:
            print("\nKonnte die Vorhersage nicht aus der Antwort lesen.")

    except requests.exceptions.ConnectionError:
        print(f"\nFEHLER: Konnte keine Verbindung zu {api_url} herstellen.")
        print("Hast du den 'mlflow models serve' oder 'docker run' Befehl in einem separaten Terminal gestartet?")
    except requests.exceptions.HTTPError as e:
        print(f"\nHTTP FEHLER von der API: {e}")
        print("Antwort der API:", e.response.text)
    except Exception as e:
        print(f"\nEin anderer Fehler ist aufgetreten: {e}")
else:
    print("Testdaten (X_test) nicht gefunden. Bitte führe die Zellen oben aus, um sie zu erstellen.")

## 🔄 8. Von manuellen Schritten zur MLOps-Pipeline

Die Schritte, die wir in diesem Workshop durchgeführt haben, können in einer professionellen Umgebung automatisiert werden. Eine strukturierte MLOps-Pipeline hilft dabei, den Prozess reproduzierbar und skalierbar zu machen.

### Vorteile einer MLOps-Pipeline:
- **Reproduzierbarkeit**: Konsistente Ausführung durch Automatisierung
- **Konfigurierbarkeit**: Parameter zentral verwalten (ähnlich zu DVC)
- **Skalierbarkeit**: Einfache Anwendung auf größere Datensätze
- **Versionierung**: Automatisches Tracking aller Artefakte und Parameter

### Beispiel einer einfachen Pipeline-Struktur:

```python
stages:
  load_data:
    cmd: python src/load_data.py
    deps:
      - src/load_data.py
    outs:
      - data/iris.csv

  train_model:
    cmd: python src/train_model.py
    deps:
      - src/train_model.py
      - data/iris.csv
      - params.yaml
    outs:
      - model.pkl
      - data/X_test.csv
      - data/y_test.csv

  evaluate:
    cmd: python src/evaluate.py
    deps:
      - src/evaluate.py
      - model.pkl
      - data/X_test.csv
      - data/y_test.csv
    metrics:
      - metrics.txt

```

In [61]:
HTML('<img src="https://raw.githubusercontent.com/asalah6/mlops-workshop/refs/heads/main/dvs_pipeline.png" width="800" height="400" alt="MLflow Tracking Visualization">')

## ✅ Geschafft! Dein erster vollständiger MLOps-Durchlauf

### **Was wurde erreicht?**
1. **Modelltraining**
   - Ein KI-Modell erfolgreich trainiert.
2. **MLflow Tracking**
   - Parameter, Metriken und Artefakte (z. B. Confusion Matrix) geloggt.
   - Vergleichbare und reproduzierbare Experimente erstellt.
3. **MLflow Model Registry**
   - Modell versioniert und Stages zugewiesen.
   - Zentrale Ablage und Vorbereitung für Produktion.
4. **Deployment-Konzepte**
   - Grundprinzipien für:
     - `mlflow models serve` (lokaler Server)
     - Bereitstellung via **Docker**.
   - Grundlagen des produktnahen Deployments verstanden.
5. **Predictions via REST API**
   - Vorhersagen mit `curl` oder `requests` abgerufen.
   - Input/Output-Formate von MLflow-APIs kennengelernt.

---

### **Warum ist das wichtig?**
- Nachvollziehbarer und reproduzierbarer ML-Workflow.
- Teamfreundlich und skalierbar.
- Bereit für professionelle MLOps-Pipelines.


---

## 🔧 Wie geht’s weiter? – Deine **Next Steps**

> Dieser Workshop war ein Einstieg – jetzt kannst du tiefer einsteigen und praxisrelevante Erweiterungen ausprobieren:

### 🖥️ 1. **MLflow UI erkunden**
  - Starte die UI lokal (`mlflow ui`) oder auf einem Server.
  - Schau dir Runs, Artefakte und die Registry im Detail an.

### ☁️ 2. **Deployment auf Cloud-Plattformen**
- MLflow lässt sich mit Cloud-Diensten kombinieren:
  - AWS SageMaker
  - Azure ML
  - Google Cloud AI Platform

### 🧪 3. **Andere Modelle und Use Cases**
- Nutze MLflow für andere Algorithmen, Frameworks und Datensätze.
- Teste Tracking und Registry mit NLP, Zeitreihen, Deep Learning usw.

### 🐳 4. **Docker-Deployment real umsetzen**
- Erstelle ein Docker-Image deines Modells und teste lokal oder in der Cloud.
- Schicke REST-Requests für echte Vorhersagen.

### 🔁 5. **CI/CD für ML aufbauen**
- Automatisiere Trainings-, Test-, Registrierungs- und Deploymentprozesse:
  - z. B. mit GitHub Actions, GitLab CI, Jenkins, Argo Workflows

---

## 🙌 Danke!

Vielen Dank für deine Teilnahme am Workshop!  
Du hast die Grundlagen gelegt für strukturiertes, nachhaltiges und teamfähiges Machine Learning – also **echtes MLOps**.

> Bleib neugierig, probiere Dinge aus und denk daran:  
> Ein gutes Modell ist nichts ohne ein gutes Deployment! 😉

---
